In [1]:
!nvidia-smi

Wed Jul  7 17:03:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   58C    P8     5W /  N/A |   2232MiB /  6144MiB |    ERR!      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import os
import pandas as pd

import sys
sys.path.append('..')

from deepmatcher.data.custom_dataset import DeepMatcherDataset
from deepmatcher.data.encoder.text_encoders import HFTextEncoder
# from deepmatcher.data.encoder.image_encoders import DINOImageEncoder
from deepmatcher.optim import Optimizer

from torch.utils.data import DataLoader
from deepmatcher.models.core import MatchingModel

In [5]:
dataset_dir = "sample_data/wdc/"
train_df = pd.read_json(os.path.join(dataset_dir, 'train.json.gz'), compression='gzip', lines=True)
print('train:', train_df.shape)
val_df = pd.read_json(os.path.join(dataset_dir, 'val.json.gz'), compression='gzip', lines=True)
print('val:', val_df.shape)
test_df = pd.read_json(os.path.join(dataset_dir, 'train.json.gz'), compression='gzip', lines=True)
print('test:', test_df.shape)

# for df in (train_df, val_df, test_df):
#     df['left_image'] = df['right_image'] = os.path.join(dataset_dir, 'test_img.jpg')
train_df.head()

train: (33943, 22)
val: (8486, 22)
test: (33943, 22)


,left_id,left_title,left_description,left_brand,left_price,left_specTableContent,left_keyValuePairs,left_category,left_cluster_id,left_identifiers,...,right_description,right_brand,right_price,right_specTableContent,right_keyValuePairs,right_category,right_cluster_id,right_identifiers,label,pair_id
0,8305739,interactive sneaker hogan women shoes brunaros...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,8305739#2447385
1,11265724,hogan interactive lav h spezzata ricamo hxw00n...,sneakers in nabuk con allacciatura frontale et...,hogan,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,11265724#8305739
2,6987609,hogan interactive sneaker in dark blue suede,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,6987609#6638421
3,6638421,hogan sneakers interactive aus wildleder in na...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,6638421#8305739
4,6987609,hogan interactive sneaker in dark blue suede,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,6987609#2447385


In [6]:
text_encoder = HFTextEncoder('albert-base-v2', trainable=False, seqtovec='mean')
# image_encoder = DINOImageEncoder(device='cuda')


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
label_column = 'label'
text_columns = ['title']
image_col = ''
tokenizer = text_encoder.tokenizer

def get_dataloader(data_df, label_column, text_cols, imaeg_col, tokenizer, batch_size, shuffle, num_workers):
    dataset = DeepMatcherDataset(
        data_df=data_df,
        label_col=label_column,
        text_cols=text_columns,
        image_col=image_col,
        tokenizer=tokenizer
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=dataset.collate_fn)

batch_size = 16
num_workers = 4

train_dataloader = get_dataloader(train_df, label_column, text_columns, image_col, tokenizer, batch_size, True, num_workers)
val_dataloader = get_dataloader(val_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)
test_dataloader = get_dataloader(test_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)

received data_df : (33943, 22)
supplied text columns : title
supplied image column : 
supplied label column : label
received data_df : (8486, 22)
supplied text columns : title
supplied image column : 
supplied label column : label
received data_df : (33943, 22)
supplied text columns : title
supplied image column : 
supplied label column : label


In [8]:
model = MatchingModel(
    text_encoder=text_encoder,
    # image_encoder=image_encoder,
    attr_summarizer=None,
    attr_condense_factor='auto',
    attr_comparator='concat',
    attr_merge='concat',
    classifier='2-layer-highway',
    hidden_size=300,
    text_attrs=text_columns,
    # image_attr='image'
    )

text_attrs: ['title']
image_attr: 
prefixes: ('left_', 'right_')
attrs: ['title']
all attrs: ['left_title', 'right_title']


In [9]:
model.initialize(next(iter(train_dataloader))['attrs'])
model

MatchingModel(
  (text_encoder): HFTextEncoder(
    (model): AlbertModel(
      (embeddings): AlbertEmbeddings(
        (word_embeddings): Embedding(30000, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (encoder): AlbertTransformer(
        (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
        (albert_layer_groups): ModuleList(
          (0): AlbertLayerGroup(
            (albert_layers): ModuleList(
              (0): AlbertLayer(
                (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (attention): AlbertAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                

In [8]:
optimizer = Optimizer(lr=1e-3, lr_decay=1e-1, start_decay_at=3)

In [10]:
model.run_train(
    train_dataloader,
    val_dataloader,
    best_save_path='best_mode.pth',
    epochs=15,
    )

* Number of trainable parameters: 1277002
===>  TRAIN Epoch 1
/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16
Finished Epoch 1 || Run Time:   15.9 | Load Time:    1.1 || F1:  28.28 | Prec:  63.64 | Rec:  18.18 || Ex/s:  18.97

===>  EVAL Epoch 1
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03
Finished Epoch 1 || Run Time:    4.8 | Load Time:    0.5 || F1:  57.78 | Prec:  61.90 | Rec:  54.17 || Ex/s:  20.30

Updating Learning Rate: 8.000e-04 Acc: 57.77778244018555 Epoch: 1
* Best F1: tensor(57.7778, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
0% [██  ] 100% | ETA: 00

KeyboardInterrupt: 

In [ ]:
model.run_eval(test_dataset)